# 2. Modelos de distribución multivariados

<p align="right">
Autor: Emmanuel Alcalá
<br>
<a href="https://scholar.google.com.mx/citations?hl=en&user=3URusCgAAAAJ&view_op=list_works&sortby=pubdate">Google Scholar</a>
</p>

<p align="left">
<br>
<a href="https://jealcalat.github.io/Analisis_multivariado/">Regresar a la página del curso</a>
</p>

---

## 2.0. Preliminares

### Eigenvalores y eigenvectores

Considerar la matriz cuadrada $A_{p\times p}$. Si existe un escalar $\lambda$ y un vector $\mathbf{x}$ tal que

$$
  A\mathbf{x} = \lambda\mathbf{x} \tag{1}
$$

En donde $\lambda$ es un _eigenvalor_ (valor propio) y $\mathbf{x}$ un eigenvector.


### Inversa

Considerar la matriz cuadrada $A_{p\times p}$. Sea la matriz cuadrada $B_{p\times p}$ una matriz con la propiedad de $AB = I = BA$. En este caso, $B_{p\times p}$ es llamada _la inversa_ de $A$, y se denota como $A^{-1}$. La matrix $I$ es una matriz especial cuyas diagonales son 1, y por encima y debajo de las diagonales solo hay 0s.

$$
  I_{p\times p}= \begin{bmatrix} 
  1 & 0 & 0 & \cdots & 0\\ 
  0 & 1 & 0  & \cdots & 0 \\
  0 & 0 & 1 & \cdots & 0 \\
  \vdots & \vdots & \vdots & \ddots & \vdots \\
  0 & 0 & 0 & \cdots & 1
  \end{bmatrix}
$$

Considerar las matrices

$$
  A = \begin{bmatrix}
  a_{11} & a_{12} \\
  a_{21} & a_{22}
  \end{bmatrix}\quad
  A' = \begin{bmatrix}
  a_{22} & -a_{12} \\
  -a_{21} & a_{11}
  \end{bmatrix}
$$

El producto de ambas es

$$
  AA' = \begin{bmatrix}
    a_{11}a_{22} - a_{12}a_{21} & 0 \\
    0 & a_{11}a_{22} - a_{12}a_{21}
  \end{bmatrix} = (a_{11}a_{22} - a_{12}a_{21})I
$$

Dado que $AA^{-1} = I$, entonces

$$
  A^{-1} = \frac{1}{a_{11}a_{22} - a_{12}a_{21}}A' = \frac{1}{a_{11}a_{22} - a_{12}a_{21}}\begin{bmatrix}
  a_{22} & -a_{12} \\
  -a_{21} & a_{11}
  \end{bmatrix}
$$

**Nota**: para que $A^{-1}$ exista, es decir, para que $A$ sea *invertible*, $a_{11}a_{22} - a_{12}a_{21} \neq 0$. 


### Determinante

Los determinantes solo están definidos para matrices cuadradas $A_{p\times p}$ (matrices que tienen el mismo número de filas que de columnas). El determinante $\text{det}(A)$ o $|A|$ es una función que mapea $A$ a números reales.

De la sección [Inversa](#inversa), sabemos que para una matriz de $2\times 2$, $A^{-1}$ existe si y solo si $a_{11}a_{22} - a_{12}a_{21} \neq 0$. Esta cantidad es el determinante de $A\in \mathbb{R}^{2\times 2}$, es decir

$$
  \text{det}(A)_{2\times 2} = a_{11}a_{22} - a_{12}a_{21}\tag{2}
$$

**Nota**: Tanto en [Inversa](#inversa) como en [Determinante](#determinante) se usó un ejemplo de una matriz sencilla de $2\times 2$. En general, para cualquier matriz $A_{p\times p}$, para que $A^{-1}$ exista, $\text{det}(A_{p\times p})\neq 0$. Es decir, *una matriz cuadrada es invertible **si y solo si** su determinante es diferente de 0.*

Se da ahora una expresión general para obtener el determinante de cualquier matriz cuadrada. Sea $A$ la matriz cuadrada ${p\times p}$. Para todo $j=1, \dots, p$, el determinante expandido a través de la fila $j$ se obtiene con

$$
  \text{det}(A) = |A| = \sum_{k=1}^{p}(-1)^{k+j}a_{jk}\text{det}(A_{j,k}) \tag{3}
$$

En donde $A_{j,k}\in \mathbb{R}^{(p-1)\times (p-1)}$ es una *submatriz* que obtenemos eliminando la fila $j$ y la columna $k$, y $a_{jk}$ es la entrada de la matriz $A$ en la fila $j$ y columna $k$. A la ecuación (2) se le conoce como la expansión de Laplace.

Por ejemplo

$$
  A = \begin{bmatrix}
  1 & 2 & 3\\
  3 & 1 & 2\\
  0 & 0 & 1
  \end{bmatrix}
$$

Usando la expansión de Laplace a en la primera fila, aplicando (2) obtenemos

$$
  |A| = \begin{vmatrix}
  1 & 2 & 3\\
  3 & 1 & 2\\
  0 & 0 & 1
  \end{vmatrix} = %
  \underbrace{(-1)^{1+1}\times 1 \begin{vmatrix}
  1 & 2\\
  0 & 1
  \end{vmatrix}}_{k=1} %
  + \underbrace{(-1)^{1+2}\times 2\begin{vmatrix}
  3 & 2\\
  0 & 1
  \end{vmatrix}}_{k=2} %
    + \underbrace{(-1)^{1+3}\times 3\begin{vmatrix}
  3 & 1\\
  0 & 0
  \end{vmatrix}}_{k=3}
$$

Podemos ahora calcular los determinantes $2\times 2$ usando la ecuación (2). Por ejemplo, 

$$
  \begin{vmatrix}
  1 & 2\\
  0 & 1
  \end{vmatrix} = 1\times 1 - 0\times 2 = 1-0
$$

Con lo que tenemos al final $\text{det}(A) = -5$.

En `R`

In [7]:
A <- matrix(
  c(
    1, 2, 3,
    3, 1, 2,
    0, 0, 1
  ),
  byrow = TRUE, # especificar que el orden sea por fila de 3 en 3
  ncol = 3
)
# el determinante
det(A)
# la inversa se obtiene con 
solve(A)

[1] -5

-2.8,-2.2,-0.6
-2.2,-2.8,-0.4
-0.6,-0.4,-0.2


-0.2,0.4,-0.2
0.6,-0.2,-1.4
0.0,0.0,1.0


### Núcleo (kernel o espacio nulo)

Considerar la siguiente ecuación

$$
  A \mathbf{x} = \mathbf{0}
$$

El kernel $\text{ker}(A\mathbf{x})$ captura todas las posibles combinaciones lineales $a_1x_1 + \dots + a_px_p$ que producen $\mathbf{0}$


### Descomposición expectral

La matriz cuadrada $A_{p\times p}$ puede se reescrita como

$$
  A = \mathbf{x} \Lambda \mathbf{x}^\top \tag{3}
$$

En donde la matriz diagonal $\Lambda = \text{diag}(\lambda_1, \lambda_2,\dots,\lambda_p)$

Un eigenvalor $\lambda$ es la raíz del polinomio de orden $p$,

$$
  (A - \lambda \mathbf{I})\mathbf{x}=\mathbf{0} \tag{4}
$$

por lo que hay $p$ eigenvalores de $A$. A cada eigenvalor $\lambda_j$ le corresponde un eigenvector $x_j$. Dado que $\mathbf{x} \neq \mathbf{0}$


## 2.2. Distribución multinormal

Ya vimos que para la distribución normal univariada, la función de densidad está dada por

$$
  p(x) = \frac{1}{\sigma \sqrt{2\pi}} e\left\{\frac{-(x-\mu)^2}{2\sigma^2}\right\}  \tag{1}
$$
